In [ ]:
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install datasets
!pip install -U sentence-transformers
!pip install POT
!pip install nltk
!python -m nltk.downloader punkt
!pip install rouge



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]

In [ ]:
from transformers import AutoTokenizer, AutoModel, T5ForConditionalGeneration, T5Tokenizer
import torch
import datasets
import statistics
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords
from nltk import download
import gensim.downloader as api
from gensim.models import KeyedVectors
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from transformers import GPT2Tokenizer, GPT2LMHeadModel

### Load Similarity model to check accuracy
Documentation:https://huggingface.co/tasks/sentence-similarity

In [ ]:

similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

"""
# predicted_answer: List of Predicted answers
# true_answers: List of corresponding true answers
"""
def sentences_similarity(predicted_answers, true_answers):
  similarity = []
  for i in range(len(predicted_answers)):
    sentences = [predicted_answers[i],true_answers[i]]
    #Compute embedding for both lists
    embedding_1= similarity_model.encode(sentences[0], convert_to_tensor=True)
    embedding_2 = similarity_model.encode(sentences[1], convert_to_tensor=True)
    score = util.pytorch_cos_sim(embedding_1, embedding_2)
    score = score.item()
    similarity.append(score)
  average = statistics.mean(similarity)
  return average



###Function for model evaluation


In [ ]:
def model_evalutate(model,dataset,count):
  counter = 0
  predict_arr = []
  true_arr = []
  for example in dataset:
    context = example['context']
    questions = example['question']
    answers = example['answers']
    # Check if there is at least one answer
    if answers and len(answers['text'])>0:        
        counter += 1
        answer_text = answers['text'][0]
        
        question_pred = model(answer_text,context)
        # Extract the answer text and start/end positions
        predict_arr.append(question_pred)
        # print(answer_text)
        # print(context)

        # print(questions)

        # print(question_pred)
        # print("---------------------------------------------------")
        true_arr.append(questions)
        if counter >= count:
          break
  return (true_arr,predict_arr)

###Load Our tuned t5-model

In [ ]:
our_en_tokenizer = T5Tokenizer.from_pretrained("ZihanXie/QuestionGeneration")
our_en_model = T5ForConditionalGeneration.from_pretrained("ZihanXie/QuestionGeneration")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
our_en_model.to(device)


def our_en_run_model(answer, context, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = 'answer:' + answer+" context:" + context
  input_string = '%s </s>' % (input_string)
  input_ids = our_en_tokenizer.encode(input_string, return_tensors="pt")
  res = our_en_model.generate(input_ids.to(device), **generator_args)
  output = our_en_tokenizer.batch_decode(res, skip_special_tokens=True)
  return output[0]

output = our_en_run_model('Gross domestic product ','GDP means gross domestic product.')
print(output)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


What does GDP stand for?


### Load Base finetuned t5-model

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

t5_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
t5_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

def t5_run_model(answer, context, max_length=520):
  
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = t5_tokenizer([input_text], return_tensors='pt')

  output = t5_model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return t5_tokenizer.decode(output[0])[16:-4]

context ="""The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."""

answer ="France"

t5_run_model(answer, context)



The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


'Where is Normandy located?'

### Load Base mt5 model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

mt5_tokenizer = AutoTokenizer.from_pretrained("Narrativa/mT5-base-finetuned-tydiQA-question-generation")

mt5_model = AutoModelForSeq2SeqLM.from_pretrained("Narrativa/mT5-base-finetuned-tydiQA-question-generation")


In [ ]:
def mt5_run_model(answer, context, max_length=520):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = mt5_tokenizer([input_text], return_tensors='pt')

  output = mt5_model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return mt5_tokenizer.decode(output[0])[6:-4]

context =""" Sofía has a degree in Communications and public relations agency experience where she was in charge of monitoring and managing PR strategy including relations with the media and journalists."""

answer ="monitoring and managing PR strategy including relations with the media and journalists"

mt5_run_model(answer, context)

'What was Sofía Martínez-Avila responsible for PR?'

In [ ]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2",pad_token="<pad>")
gpt2_model = GPT2LMHeadModel.from_pretrained("danyaljj/gpt2_question_generation_given_paragraph_answer",  pad_token_id=gpt2_tokenizer.eos_token_id)

def gpt2_run_model(answer, context, max_length=512):
  # input_text = "answer: %s  context: %s </s>" % (answer, context)
  input_text = context+" A: "+ answer+" Q:"
  features = gpt2_tokenizer([input_text],  return_tensors='pt')
  output = gpt2_model.generate(input_ids=features['input_ids'], attention_mask=features['attention_mask'],max_length=max_length)
  # input_ids = tokenizer.encode(input_str, return_tensors="pt")
  # print(input_ids)
  # print(input_str)
  # outputs = model.generate(input_ids,max_length=max_length)
  return (gpt2_tokenizer.decode(output[0], skip_special_tokens=True)[len(input_text):])
print(gpt2_run_model("apples", "There are two apples on the counter."))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 What is the name of the item that is on the counter?


### Load testing data
squad_v2: https://huggingface.co/datasets/GEM/squad_v2/viewer/gem_data_split/test  
cmrc2018:https://github.com/ymcui/cmrc2018

In [ ]:
squad_v2 = datasets.load_dataset("GEM/squad_v2", split="test")
cmrc2018 = datasets.load_dataset('cmrc2018', split="test")

dataset = datasets.load_dataset("squad", split="validation")
"badokorach/NewQA"
squadv2 = datasets.load_dataset("squad_v2", split="validation")


### Validation on our model


###EN our

In [ ]:
our_en_true, our_en_pred = model_evalutate(our_en_run_model,squad_v2,300 )
print(len(our_en_true),(len(our_en_pred)))
average = sentences_similarity(our_en_true, our_en_pred)
print("The average of our EN model:", average)

300 300
The average of our EN model: 0.8001428450768192


### EN T5



In [ ]:
t5_true, t5_pred = model_evalutate(t5_run_model,squad_v2,300 )
average = sentences_similarity(t5_true, t5_pred)
print("The average of t5 model:", average)

The average of t5 model: 0.8193745869894823


### EN mt5

In [ ]:
mt5_en_true, mt5_en_pred = model_evalutate(mt5_run_model,squad_v2,300 )
average = sentences_similarity(mt5_en_true, mt5_en_pred)
print("The average of mt5 model on EN:", average)

The average of mt5 model on EN: 0.5601809611637145


### EN GPT2

In [ ]:
gpt2_en_true, gpt2_en_pred = model_evalutate(gpt2_run_model,squad_v2,300 )
average = sentences_similarity(gpt2_en_true, gpt2_en_pred)
print("The average of gpt2 model on EN:", average)

The average of gpt2 model on EN: 0.7408919122070075


### Word Mover Distance Model
A lower WMD value indicates higher similarity between the texts, while a higher WMD value indicates lower similarity or higher dissimilarity between the texts.

Documentation:https://radimrehurek.com/gensim/auto_examples/tutorials/run_wmd.html

In [ ]:
# Import and download stopwords from NLTK.
word2vec_model = api.load('word2vec-google-news-300') 
download('stopwords') 
stop_words = stopwords.words('english')
def preprocess(sentence):
    return [w for w in sentence.lower().split() if w not in stop_words]

[==================================================] 100.0% 1662.8/1662.8MB downloaded


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def WMD(sentence1, sentence2):

  processed_s1 = preprocess(sentence1)
  processed_s2 = preprocess(sentence2)
  distance = word2vec_model.wmdistance(processed_s1, processed_s2)
  # print('distance = %.4f' % distance)
  return distance


In [ ]:
import math
def model_WMD(true_answers, predicted_answers):
  distance = []
  for i in range(len(predicted_answers)):
    #Compute embedding for both lists
    
    wmd = WMD(true_answers[i], predicted_answers[i])
    if math.isinf(wmd):
      continue
    else:
      distance.append(wmd)
  average = statistics.mean(distance)
  return average

In [ ]:
our_wmd = model_WMD(our_en_true, our_en_pred)
print("Our model WMD: ",our_wmd)



Our model WMD:  0.5985124385375024


In [ ]:
t5_wmd = model_WMD(t5_true, t5_pred)
print("t5 model WMD: ",t5_wmd)

t5 model WMD:  0.5983119764177492


In [ ]:
mt5_wmd = model_WMD(mt5_en_true, mt5_en_pred)
print("mt-5 model WMD: ",mt5_wmd)

mt-5 model WMD:  1.0819367661411814


In [ ]:
gpt2_wmd = model_WMD(gpt2_en_true, gpt2_en_pred)
print("gpt2 model WMD: ",gpt2_wmd)

gpt2 model WMD:  0.7264586745445931


In [ ]:
def get_BLEU_score_en(true_answers, predicted_answers):
  scores = []
  for i in range(len(predicted_answers)):
    #Compute embedding for both lists

    tokens1 = word_tokenize(true_answers[i])
    tokens2 = word_tokenize(predicted_answers[i])

    bleu_score = sentence_bleu([tokens1], tokens2)
    scores.append(bleu_score)
  average = statistics.mean(scores)
  return average

In [ ]:

our_en_bleu = get_BLEU_score_en(our_en_true, our_en_pred)
t5_en_bleu = get_BLEU_score_en(t5_true, t5_pred)
mt5_en_bleu = get_BLEU_score_en(mt5_en_true, mt5_en_pred)
gpt2_en_bleu = get_BLEU_score_en(gpt2_en_true, gpt2_en_pred)

print("our en BLEU:",our_en_bleu)
print("t5 en BLEU:",t5_en_bleu)
print("mt5 en BLEU:",mt5_en_bleu)
print("gpt2 en BLEU:",gpt2_en_bleu)

our en BLEU: 0.31606019515298467
t5 en BLEU: 0.32948509830950196
mt5 en BLEU: 0.03622468974179872
gpt2 en BLEU: 0.24380164090843812


In [ ]:
from rouge import Rouge
rouge = Rouge()
# Compute ROUGE-1, ROUGE-2, and ROUGE-L scores for sentences1 and sentences2 separately
scores1 = rouge.get_scores(our_en_true, our_en_pred, avg=True)
# print(scores1)
# Print the scores
print("rouge_our")
for i, s in enumerate(scores1):
    print(f"ROUGE-{i+1}: {s}")
    print(scores1[s])


rouge_our
ROUGE-1: rouge-1
{'r': 0.5957828952809604, 'p': 0.5442610569398497, 'f': 0.5591902924410386}
ROUGE-2: rouge-2
{'r': 0.4297418550947963, 'p': 0.3935395678939796, 'f': 0.4027162346350833}
ROUGE-3: rouge-l
{'r': 0.5748032624410335, 'p': 0.5255058300116353, 'f': 0.5398041494509297}


In [ ]:
rouge_t5 = rouge.get_scores(t5_true, t5_pred, avg=True)
# print(scores1)
# Print the scores\
print("rouge_t5")

for i, s in enumerate(rouge_t5):
    print(f"ROUGE-{i+1}: {s}")
    print(rouge_t5[s])


rouge_t5
ROUGE-1: rouge-1
{'r': 0.5939308800218244, 'p': 0.5617119601101805, 'f': 0.5686279245240401}
ROUGE-2: rouge-2
{'r': 0.4231218843674086, 'p': 0.40781975633299145, 'f': 0.4086386558962401}
ROUGE-3: rouge-l
{'r': 0.5731601619055181, 'p': 0.5439793957046282, 'f': 0.5499712815412329}


In [ ]:
rouge_mt5 = rouge.get_scores(mt5_en_true, mt5_en_pred, avg=True)
# print(scores1)
# Print the scores
print("rouge_mt5")

for i, s in enumerate(rouge_mt5):
    print(f"ROUGE-{i+1}: {s}")
    print(rouge_mt5[s])


rouge_mt5
ROUGE-1: rouge-1
{'r': 0.31205278980279, 'p': 0.2371024282448432, 'f': 0.2622402134278826}
ROUGE-2: rouge-2
{'r': 0.10727248677248674, 'p': 0.08820368656398068, 'f': 0.09405312699455814}
ROUGE-3: rouge-l
{'r': 0.2956730399230401, 'p': 0.22651758750365572, 'f': 0.24969364226640156}


In [ ]:
rouge_gpt2 = rouge.get_scores(gpt2_en_true,gpt2_en_pred,avg=True)
# print(scores1)
# Print the scores
print("rouge_gpt2")

for i, s in enumerate(rouge_gpt2):
    print(f"ROUGE-{i+1}: {s}")
    print(rouge_gpt2[s])


rouge_gpt2
ROUGE-1: rouge-1
{'r': 0.5236675952871608, 'p': 0.47156020546074745, 'f': 0.4848521826967172}
ROUGE-2: rouge-2
{'r': 0.3483190444459798, 'p': 0.31177375772375787, 'f': 0.3211281645698845}
ROUGE-3: rouge-l
{'r': 0.5046402772892544, 'p': 0.45452173671987905, 'f': 0.4674318254930588}
